# Strava segment data
TODO: Get segment id

In [1]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

In [ ]:
#Required to activate card-section below
lt=[t for t in browser.find_elements_by_class_name('card-section') 
   if t.text.find('Latest Activity')>-1][0]

lt.find_element_by_tag_name('a').click()

# Web page with latest activity

# General info

In [11]:
import time

In [12]:
time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

browser.find_element_by_id('show-hidden-efforts').click()

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time
# Segments

sgmt=ll[1][['Name','Time','Speed','Power']]

sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]



In [ ]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [6]:
df.to_json(db_file)

In [7]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [8]:
!bash commit.sh

[master e52c016] La Tuna 2:15 PM on Saturday, May 23, 2020
 3 files changed, 20 insertions(+), 6 deletions(-)
 rewrite db/strava.json (100%)
Counting objects: 6, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 2.88 KiB | 0 bytes/s, done.
Total 6 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   6b322b4..e52c016  master -> master


In [10]:
time

'51:27'